### Anomaly Detection Mini Project
#### Corey Solitaire
`11.09.2020`

<div class="alert alert-block alert-info"><b></b></div>

#### Imports:

In [1]:

import wrangle
import explore
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
import seaborn as sns

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

<div class="alert alert-block alert-info"><b></b></div>

### Import wrangle df:

In [2]:
df = wrangle.read_csv()
df = wrangle.wrangle_df(df)
df.head(1)

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-01-26 09:55:03,/,1,8,97.105.19.61,2018,1,26,9,Friday,1634276157


In [3]:
df.tail(1)

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-11-02 16:48:47,javascript-i/bom-and-dom/bom,763,62,107.192.148.199,2020,11,2,16,Monday,1807783111


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 719459 entries, 2018-01-26 09:55:03 to 2020-11-02 16:48:47
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   page_viewed  719458 non-null  object
 1   user_id      719459 non-null  object
 2   cohort_id    674619 non-null  object
 3   ip           719459 non-null  object
 4   year         719459 non-null  object
 5   month        719459 non-null  object
 6   day          719459 non-null  object
 7   hour         719459 non-null  object
 8   weekday      719459 non-null  object
 9   int_ip       719459 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 60.4+ MB


In [5]:
df = df.fillna(0)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 719459 entries, 2018-01-26 09:55:03 to 2020-11-02 16:48:47
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   page_viewed  719459 non-null  object 
 1   user_id      719459 non-null  int64  
 2   cohort_id    719459 non-null  float64
 3   ip           719459 non-null  object 
 4   year         719459 non-null  int64  
 5   month        719459 non-null  int64  
 6   day          719459 non-null  int64  
 7   hour         719459 non-null  int64  
 8   weekday      719459 non-null  object 
 9   int_ip       719459 non-null  int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 60.4+ MB


In [7]:
cohort_id = df.cohort_id.unique().tolist()
sorted(cohort_id, reverse = True)

[62.0,
 61.0,
 59.0,
 58.0,
 57.0,
 56.0,
 55.0,
 53.0,
 52.0,
 51.0,
 34.0,
 33.0,
 32.0,
 31.0,
 29.0,
 28.0,
 27.0,
 26.0,
 25.0,
 24.0,
 23.0,
 22.0,
 21.0,
 19.0,
 18.0,
 17.0,
 16.0,
 15.0,
 14.0,
 13.0,
 12.0,
 11.0,
 9.0,
 8.0,
 7.0,
 6.0,
 5.0,
 4.0,
 2.0,
 1.0,
 0.0]

`Explore Missing user_id Further`

<div class="alert alert-block alert-info"><b></b></div>

# Questions to Answer:
****
  1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?


  2. Is there a cohort that referred to a lesson significantly more that other cohorts seemed to gloss over? 


  3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students? 


  4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses? Any odd user-agents? 


  5. At some point in the last year, ability for students and alumni to cross-access curriculum (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before? 


  6. What topics are grads continuing to reference after graduation and into their jobs (for each program)? 


  7. Which lessons are least accessed? 


  8. Anything else I should be aware of? 

<div class="alert alert-block alert-info"><b></b></div>

## First Step is to ID Cohorts:

explore.cohort_id(df, year, month, day, hour)
***

In [8]:
# 21,Sequoia,2017-09-27,2018-02-15,2
# Cohort ID #21 I think

In [9]:
# 22,Teddy,2018-01-08,2018-05-17,2 
# test = explore.cohort_id(df,2018,1,26,10)
# test.head()

#### Cohort Teddy

In [10]:
teddy = df[df.cohort_id == 22]
len(teddy.user_id.unique())

35

In [11]:
teddy.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2018,1,26,9,Friday,1634276157
2018-01-26 09:56:41,javascript-i/loops,2,22.0,97.105.19.61,2018,1,26,9,Friday,1634276157
2018-01-26 09:56:46,javascript-i/conditionals,3,22.0,97.105.19.61,2018,1,26,9,Friday,1634276157
2018-01-26 09:56:48,javascript-i/functions,3,22.0,97.105.19.61,2018,1,26,9,Friday,1634276157
2018-01-26 09:56:59,javascript-i/loops,2,22.0,97.105.19.61,2018,1,26,9,Friday,1634276157


In [12]:
# 23,Ulysses,2018-03-05,2018-07-19,2
# test = explore.cohort_id(df,2018,3,5,10)
# test.head()

#### Cohort Ulysses

In [13]:
ulysses = df[df.cohort_id == 23]
len(ulysses.user_id.unique())

33

In [14]:
ulysses.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-03-05 14:22:48,html-css/css-i,25,23.0,97.105.19.61,2018,3,5,14,Monday,1634276157
2018-03-05 14:22:58,html-css/forms,25,23.0,97.105.19.61,2018,3,5,14,Monday,1634276157
2018-03-05 14:23:06,html-css/css-i,25,23.0,97.105.19.61,2018,3,5,14,Monday,1634276157
2018-03-05 14:23:08,html-css/css-i/introduction,25,23.0,97.105.19.61,2018,3,5,14,Monday,1634276157
2018-03-05 14:23:47,html-css,25,23.0,97.105.19.61,2018,3,5,14,Monday,1634276157


In [15]:
# 24,Voyageurs,2018-05-29,2018-10-11,2
# test = explore.cohort_id(df,2018,5,29,10)
# test.head()

#### Cohort Voyagers

In [16]:
voyagers = df[df.cohort_id == 24]
len(voyagers.user_id.unique())

34

In [17]:
voyagers.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-05-29 14:42:38,/,182,24.0,97.105.19.61,2018,5,29,14,Tuesday,1634276157
2018-05-29 14:42:39,/,183,24.0,97.105.19.61,2018,5,29,14,Tuesday,1634276157
2018-05-29 14:42:42,/,184,24.0,97.105.19.61,2018,5,29,14,Tuesday,1634276157
2018-05-29 14:42:42,/,185,24.0,97.105.19.61,2018,5,29,14,Tuesday,1634276157
2018-05-29 14:42:42,/,186,24.0,97.105.19.61,2018,5,29,14,Tuesday,1634276157


In [18]:
# 25,Wrangell,2018-07-23,2018-11-29,2
# test = explore.cohort_id(df,2018,7,23,10)
# test.head()

#### Cohort Wranglell

In [19]:
wranglell = df[df.cohort_id == 25]
len(wranglell.user_id.unique())

26

In [20]:
wranglell.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-07-23 13:51:26,registerUser,218,25.0,97.105.19.58,2018,7,23,13,Monday,1634276154
2018-07-23 13:51:39,registerUser,218,25.0,97.105.19.58,2018,7,23,13,Monday,1634276154
2018-07-23 15:15:19,/,219,25.0,97.105.19.58,2018,7,23,15,Monday,1634276154
2018-07-23 15:15:20,/,220,25.0,97.105.19.58,2018,7,23,15,Monday,1634276154
2018-07-23 15:15:21,/,221,25.0,97.105.19.58,2018,7,23,15,Monday,1634276154


In [21]:
# 26,Xanadu,2018-09-17,2019-02-08,2
# test = explore.cohort_id(df,2018,9,17,10)
# test.head()

#### Cohort Xanadu

In [22]:
xanadu = df[df.cohort_id == 26]
len(xanadu.user_id.unique())

20

In [23]:
xanadu.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-09-17 14:12:28,/,259,26.0,97.105.19.58,2018,9,17,14,Monday,1634276154
2018-09-17 14:13:38,/,260,26.0,97.105.19.58,2018,9,17,14,Monday,1634276154
2018-09-17 14:13:42,html-css,260,26.0,97.105.19.58,2018,9,17,14,Monday,1634276154
2018-09-17 14:13:58,/,261,26.0,97.105.19.58,2018,9,17,14,Monday,1634276154
2018-09-17 14:14:21,/,262,26.0,97.105.19.58,2018,9,17,14,Monday,1634276154


In [24]:
# 27,Yosemite,2018-11-05,2019-04-03,2
# test = explore.cohort_id(df,2018,11,5,10)
# test.head(20)

#### Cohort Yosemite

In [25]:
yosemite = df[df.cohort_id == 27]
len(yosemite.user_id.unique())

29

In [26]:
yosemite.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-11-05 14:03:58,/,287,27.0,97.105.19.58,2018,11,5,14,Monday,1634276154
2018-11-05 14:04:58,/,288,27.0,97.105.19.58,2018,11,5,14,Monday,1634276154
2018-11-05 14:05:18,/,289,27.0,97.105.19.58,2018,11,5,14,Monday,1634276154
2018-11-05 14:06:41,/,290,27.0,97.105.19.58,2018,11,5,14,Monday,1634276154
2018-11-05 14:06:51,/,291,27.0,97.105.19.58,2018,11,5,14,Monday,1634276154


****

In [27]:
# 28,Staff,2014-02-04,2014-02-04,2
staff = df[df.cohort_id == 28]
len(staff.user_id.unique())

35

In [28]:
staff.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2018-12-06 14:15:29,appendix/further-reading/java/intellij-tomcat-...,53,28.0,97.105.19.58,2018,12,6,14,Thursday,1634276154
2018-12-06 14:25:40,javascript-i,314,28.0,97.105.19.58,2018,12,6,14,Thursday,1634276154
2018-12-06 14:25:41,javascript-i/coffee-project,314,28.0,97.105.19.58,2018,12,6,14,Thursday,1634276154
2018-12-06 15:31:29,spring,40,28.0,170.248.173.247,2018,12,6,15,Thursday,2868424183
2018-12-06 15:31:33,spring/fundamentals/security,40,28.0,170.248.173.247,2018,12,6,15,Thursday,2868424183


`Staff are listed under the webdev cohorts (2)`

****

In [29]:
# 29,Zion,2019-01-22,2019-06-04,2
test = explore.cohort_id(df,2019,1,22,9)
test.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-01-22 09:02:00,spring,272,26.0,97.105.19.58,2019,1,22,9,Tuesday,1634276154
2019-01-22 09:02:05,spring/fundamentals/form-model-binding,272,26.0,97.105.19.58,2019,1,22,9,Tuesday,1634276154
2019-01-22 09:02:23,java-ii/annotations,295,27.0,97.105.19.58,2019,1,22,9,Tuesday,1634276154
2019-01-22 09:02:37,java-ii/annotations,289,27.0,97.105.19.58,2019,1,22,9,Tuesday,1634276154
2019-01-22 09:02:38,java-ii/collections,292,27.0,97.105.19.58,2019,1,22,9,Tuesday,1634276154


#### Cohort Zion

In [30]:
zion = df[df.cohort_id == 29]
len(zion.user_id.unique())

25

In [31]:
zion.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-01-22 15:09:54,/,323,29.0,97.105.19.58,2019,1,22,15,Tuesday,1634276154
2019-01-22 15:10:01,toc,323,29.0,97.105.19.58,2019,1,22,15,Tuesday,1634276154
2019-01-22 15:11:47,/,324,29.0,97.105.19.58,2019,1,22,15,Tuesday,1634276154
2019-01-22 15:11:53,toc,323,29.0,97.105.19.58,2019,1,22,15,Tuesday,1634276154
2019-01-22 15:11:55,/,325,29.0,97.105.19.58,2019,1,22,15,Tuesday,1634276154


In [32]:
# zion_id = test.user_id.unique().tolist()
# sorted(zion_id, reverse = True)

In [33]:
# 30,Ada,2019-02-04,2019-06-16,3
# test = explore.cohort_id(df,2019,2,4,9)
# test.tail(5)

#### Cohort Ada

`To determine Ada`
1. Identified Ada to be missing cohort_id, correct time span returned Nan
2. 75 user_id were missing cohort_id
3. Sorted user_id untill found id's that matched time period

In [34]:
ada = df[df.cohort_id == 0]
len(ada.user_id.unique())

75

In [35]:
# To find ada must find not ada first
ada = ada[ada.user_id != 48]
ada = ada[ada.user_id != 54]
ada = ada[ada.user_id != 58]
ada = ada[ada.user_id != 59]
ada = ada[ada.user_id != 61]
ada = ada[ada.user_id != 62]
ada = ada[ada.user_id != 63]
ada = ada[ada.user_id != 64]
ada = ada[ada.user_id != 73]
ada = ada[ada.user_id != 74]
ada = ada[ada.user_id != 78]
ada = ada[ada.user_id != 79]
ada = ada[ada.user_id != 86]
ada = ada[ada.user_id != 88]
ada = ada[ada.user_id != 89]
ada = ada[ada.user_id != 100]
ada = ada[ada.user_id != 103]
ada = ada[ada.user_id != 111]
ada = ada[ada.user_id != 137]
ada = ada[ada.user_id != 166]
ada = ada[ada.user_id != 176]
ada = ada[ada.user_id != 213]
ada = ada[ada.user_id != 247]
ada = ada[ada.user_id != 317]
ada = ada[ada.user_id != 782]
ada = ada[ada.user_id != 744]
ada = ada[ada.user_id != 736]
ada = ada[ada.user_id != 731]
ada = ada[ada.user_id != 729]
ada = ada[ada.user_id != 728]
ada = ada[ada.user_id != 727]
ada = ada[ada.user_id != 726]
ada = ada[ada.user_id != 725]
ada = ada[ada.user_id != 724]
ada = ada[ada.user_id != 723]
ada = ada[ada.user_id != 722]
ada = ada[ada.user_id != 721]
ada = ada[ada.user_id != 720]
ada = ada[ada.user_id != 719]
ada = ada[ada.user_id != 718]
ada = ada[ada.user_id != 717]
ada = ada[ada.user_id != 716]
ada = ada[ada.user_id != 715]
ada = ada[ada.user_id != 714]
ada = ada[ada.user_id != 713]
ada = ada[ada.user_id != 663]
ada = ada[ada.user_id != 644]
ada = ada[ada.user_id != 544]
ada = ada[ada.user_id != 429]
ada = ada[ada.user_id != 406]
ada = ada[ada.user_id != 403]
ada['cohort_id'] = ada['cohort_id'].replace(0,30)
ada.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-02-04 16:22:20,/,349,30.0,97.105.19.58,2019,2,4,16,Monday,1634276154
2019-02-04 16:22:36,/,350,30.0,97.105.19.58,2019,2,4,16,Monday,1634276154
2019-02-04 16:23:07,1._Fundamentals,350,30.0,97.105.19.58,2019,2,4,16,Monday,1634276154
2019-02-04 16:23:34,/,351,30.0,97.105.19.58,2019,2,4,16,Monday,1634276154
2019-02-04 16:36:36,/,352,30.0,97.105.19.58,2019,2,4,16,Monday,1634276154


In [36]:
len(ada.user_id.unique())

24

In [37]:
# 31,Andromeda,2019-03-18,2019-07-30,2
test = explore.cohort_id(df,2019,3,18,10)
test.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-03-18 10:00:14,Exercises,248,28.0,97.105.19.58,2019,3,18,10,Monday,1634276154
2019-03-18 10:00:51,Univariate_Regression_Excel,248,28.0,97.105.19.58,2019,3,18,10,Monday,1634276154
2019-03-18 10:03:37,Univariate_Regression_Excel,366,0.0,97.105.19.58,2019,3,18,10,Monday,1634276154
2019-03-18 10:03:42,Univariate_Regression_Excel,368,0.0,97.105.19.58,2019,3,18,10,Monday,1634276154
2019-03-18 10:06:32,Univariate_Regression_Excel,357,0.0,97.105.19.58,2019,3,18,10,Monday,1634276154


#### Cohort: Andromeda

In [38]:
andromeda = df[df.cohort_id == 31]
len(andromeda.user_id.unique())

28

In [39]:
andromeda.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-03-18 16:01:40,assets/js/pdfmake.min.js.map,373,31.0,97.105.19.58,2019,3,18,16,Monday,1634276154
2019-03-18 16:02:00,/,374,31.0,97.105.19.58,2019,3,18,16,Monday,1634276154
2019-03-18 16:02:08,toc,374,31.0,97.105.19.58,2019,3,18,16,Monday,1634276154
2019-03-18 18:53:35,/,376,31.0,173.173.127.101,2019,3,18,18,Monday,2913828709
2019-03-18 18:55:34,toc,376,31.0,173.173.127.101,2019,3,18,18,Monday,2913828709


#### Cohort Betelgeuse

In [40]:
# 32,Betelgeuse,2019-05-28,2019-10-08,2
betelgeuse = df[df.cohort_id == 32]
len(betelgeuse.user_id.unique())

21

In [41]:
betelgeuse.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-05-29 09:48:59,/,407,32.0,97.105.19.58,2019,5,29,9,Wednesday,1634276154
2019-05-29 09:56:08,/,408,32.0,97.105.19.58,2019,5,29,9,Wednesday,1634276154
2019-05-29 10:01:23,/,409,32.0,97.105.19.58,2019,5,29,10,Wednesday,1634276154
2019-05-29 10:01:28,/,410,32.0,97.105.19.58,2019,5,29,10,Wednesday,1634276154
2019-05-29 10:01:31,html-css,409,32.0,97.105.19.58,2019,5,29,10,Wednesday,1634276154


#### Cohort Ceres

In [42]:
# 33,Ceres,2019-07-15,2019-12-11,2
ceres = df[df.cohort_id == 33]
len(ceres.user_id.unique())

29

In [43]:
ceres.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-07-15 15:39:34,/,433,33.0,97.105.19.58,2019,7,15,15,Monday,1634276154
2019-07-15 15:39:48,toc,433,33.0,97.105.19.58,2019,7,15,15,Monday,1634276154
2019-07-15 15:40:29,quiz,434,33.0,97.105.19.58,2019,7,15,15,Monday,1634276154
2019-07-15 15:40:33,/,434,33.0,97.105.19.58,2019,7,15,15,Monday,1634276154
2019-07-15 15:40:35,javascript-i,433,33.0,97.105.19.58,2019,7,15,15,Monday,1634276154


#### Cohort Bayes

In [44]:
# 34,Bayes,2019-08-19,2020-01-30,3
bayes = df[df.cohort_id == 34]
len(bayes.user_id.unique())

23

In [45]:
bayes.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-08-20 09:39:58,/,466,34.0,97.105.19.58,2019,8,20,9,Tuesday,1634276154
2019-08-20 09:39:59,/,467,34.0,97.105.19.58,2019,8,20,9,Tuesday,1634276154
2019-08-20 09:39:59,/,468,34.0,97.105.19.58,2019,8,20,9,Tuesday,1634276154
2019-08-20 09:40:02,/,469,34.0,97.105.19.58,2019,8,20,9,Tuesday,1634276154
2019-08-20 09:40:08,/,470,34.0,97.105.19.58,2019,8,20,9,Tuesday,1634276154


#### Cohort Deimos

In [46]:
# 51,Deimos,2019-09-16,2020-02-27,2
deimos = df[df.cohort_id == 51]
len(deimos.user_id.unique())

27

In [47]:
deimos.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-09-16 15:45:32,/,488,51.0,97.105.19.58,2019,9,16,15,Monday,1634276154
2019-09-16 15:59:40,/,489,51.0,97.105.19.58,2019,9,16,15,Monday,1634276154
2019-09-16 15:59:46,html-css,489,51.0,97.105.19.58,2019,9,16,15,Monday,1634276154
2019-09-16 15:59:50,javascript-i,489,51.0,97.105.19.58,2019,9,16,15,Monday,1634276154
2019-09-16 15:59:52,html-css,489,51.0,97.105.19.58,2019,9,16,15,Monday,1634276154


#### Cohort Europa

In [48]:
# 52,Europa,2019-11-04,2020-04-17,2
europa = df[df.cohort_id == 52]
len(europa.user_id.unique())

28

In [49]:
europa.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2019-11-04 16:03:34,/,516,52.0,97.105.19.58,2019,11,4,16,Monday,1634276154
2019-11-04 16:03:35,/,517,52.0,97.105.19.58,2019,11,4,16,Monday,1634276154
2019-11-04 16:03:35,/,518,52.0,97.105.19.58,2019,11,4,16,Monday,1634276154
2019-11-04 16:03:35,/,519,52.0,97.105.19.58,2019,11,4,16,Monday,1634276154
2019-11-04 16:03:35,/,520,52.0,97.105.19.58,2019,11,4,16,Monday,1634276154


#### Cohort Fortuna

In [50]:
# 53,Fortuna,2020-01-13,2020-06-23,2
fortuna = df[df.cohort_id == 53]
len(fortuna.user_id.unique())

28

In [51]:
fortuna.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-01-13 16:00:03,/,120,53.0,97.105.19.58,2020,1,13,16,Monday,1634276154
2020-01-13 16:01:16,toc,120,53.0,97.105.19.58,2020,1,13,16,Monday,1634276154
2020-01-13 16:01:22,html-css/introduction,120,53.0,97.105.19.58,2020,1,13,16,Monday,1634276154
2020-01-13 16:01:28,html-css/elements,120,53.0,97.105.19.58,2020,1,13,16,Monday,1634276154
2020-01-13 16:01:43,/,547,53.0,97.105.19.58,2020,1,13,16,Monday,1634276154


#### Cohort Curie

In [52]:
# 55,Curie,2020-02-03,2020-07-07,3
curie = df[df.cohort_id == 55]
len(curie.user_id.unique())

21

In [53]:
curie.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-02-03 15:39:34,login,575,55.0,97.105.19.58,2020,2,3,15,Monday,1634276154
2020-02-03 15:39:35,/,576,55.0,97.105.19.58,2020,2,3,15,Monday,1634276154
2020-02-03 15:39:37,/,577,55.0,97.105.19.58,2020,2,3,15,Monday,1634276154
2020-02-03 15:39:37,login,575,55.0,97.105.19.58,2020,2,3,15,Monday,1634276154
2020-02-03 15:39:43,/,578,55.0,97.105.19.58,2020,2,3,15,Monday,1634276154


#### Cohort Apex

In [54]:
# 56,Apex,2020-02-24,2020-07-29,2
apex = df[df.cohort_id == 56]
len(apex.user_id.unique())

23

In [55]:
apex.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-02-24 15:23:42,/,594,56.0,12.106.208.194,2020,2,24,15,Monday,208326850
2020-02-24 15:23:43,/,595,56.0,12.106.208.194,2020,2,24,15,Monday,208326850
2020-02-24 15:23:44,/,596,56.0,12.106.208.194,2020,2,24,15,Monday,208326850
2020-02-24 15:23:48,/,597,56.0,12.106.208.194,2020,2,24,15,Monday,208326850
2020-02-24 15:23:48,/,598,56.0,12.106.208.194,2020,2,24,15,Monday,208326850


#### Cohort Ganymede

In [56]:
# 57,Ganymede,2020-03-23,2020-08-20,2
ganymead = df[df.cohort_id == 57]
len(ganymead.user_id.unique())

28

In [57]:
ganymead.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-03-23 14:10:17,/,619,57.0,97.105.19.58,2020,3,23,14,Monday,1634276154
2020-03-23 15:09:34,/,621,57.0,97.105.19.58,2020,3,23,15,Monday,1634276154
2020-03-23 15:09:41,/,622,57.0,97.105.19.58,2020,3,23,15,Monday,1634276154
2020-03-23 15:09:42,/,623,57.0,97.105.19.58,2020,3,23,15,Monday,1634276154
2020-03-23 15:09:46,/,624,57.0,97.105.19.58,2020,3,23,15,Monday,1634276154


#### Cohort Hyperion

In [58]:
# 58,Hyperion,2020-05-26,2020-11-10,2
hyperion = df[df.cohort_id == 58]
len(hyperion.user_id.unique())

22

In [59]:
hyperion.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-05-26 16:21:50,/,655,58.0,107.128.2.166,2020,5,26,16,Tuesday,1803551398
2020-05-26 17:03:59,toc,655,58.0,107.128.2.166,2020,5,26,17,Tuesday,1803551398
2020-05-26 17:04:28,html-css,655,58.0,107.128.2.166,2020,5,26,17,Tuesday,1803551398
2020-05-27 00:31:09,/,655,58.0,162.194.247.205,2020,5,27,0,Wednesday,2730686413
2020-05-27 00:31:24,/,655,58.0,162.194.247.205,2020,5,27,0,Wednesday,2730686413


#### Cohort Darden

In [60]:
# 59,Darden,2020-07-13,2021-01-12,3
darden = df[df.cohort_id == 59]
len(darden.user_id.unique())

28

In [61]:
darden.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-07-13 14:34:44,/,678,59.0,76.201.20.193,2020,7,13,14,Monday,1288246465
2020-07-13 14:37:22,/,679,59.0,24.28.146.155,2020,7,13,14,Monday,404525723
2020-07-13 14:38:28,/,680,59.0,136.50.56.155,2020,7,13,14,Monday,2284992667
2020-07-13 14:39:21,13-advanced-topics/1-tidy-data,679,59.0,24.28.146.155,2020,7,13,14,Monday,404525723
2020-07-13 14:39:36,1-fundamentals/1.1-intro-to-data-science,679,59.0,24.28.146.155,2020,7,13,14,Monday,404525723


#### Cohort Io

In [62]:
# 60,Io,2020-07-20,2021-01-21,2
io = df[df.cohort_id == 61]
len(io.user_id.unique())

22

In [63]:
io.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-07-20 14:05:55,/,700,61.0,99.43.143.116,2020,7,20,14,Monday,1663799156
2020-07-20 14:08:19,/,701,61.0,47.184.206.205,2020,7,20,14,Monday,800640717
2020-07-20 14:08:26,/,702,61.0,76.221.87.98,2020,7,20,14,Monday,1289574242
2020-07-20 14:08:29,/,703,61.0,67.11.100.169,2020,7,20,14,Monday,1124820137
2020-07-20 14:08:30,/,704,61.0,76.185.33.217,2020,7,20,14,Monday,1287201241


#### Cohort Bash

In [64]:
# 61,Bash,2020-07-20,2021-01-21,2
bash = df[df.cohort_id == 0]
bash = bash[bash.user_id != 48]
bash = bash[bash.user_id != 54]
bash = bash[bash.user_id != 58]
bash = bash[bash.user_id != 59]
bash = bash[bash.user_id != 61]
bash = bash[bash.user_id != 62]
bash = bash[bash.user_id != 63]
bash = bash[bash.user_id != 64]
bash = bash[bash.user_id != 73]
bash = bash[bash.user_id != 74]
bash = bash[bash.user_id != 78]
bash = bash[bash.user_id != 79]
bash = bash[bash.user_id != 86]
bash = bash[bash.user_id != 88]
bash = bash[bash.user_id != 89]
bash = bash[bash.user_id != 100]
bash = bash[bash.user_id != 103]
bash = bash[bash.user_id != 111]
bash = bash[bash.user_id != 137]
bash = bash[bash.user_id != 166]
bash = bash[bash.user_id != 176]
bash = bash[bash.user_id != 213]
bash = bash[bash.user_id != 247]
bash = bash[bash.user_id != 317]
bash = bash[bash.user_id != 663]
bash = bash[bash.user_id != 644]
bash = bash[bash.user_id != 544]
bash = bash[bash.user_id != 429]
bash = bash[bash.user_id != 406]
bash = bash[bash.user_id != 403]
bash = bash[bash.user_id != 349]
bash = bash[bash.user_id != 350]
bash = bash[bash.user_id != 351]
bash = bash[bash.user_id != 352]
bash = bash[bash.user_id != 353]
bash = bash[bash.user_id != 354] 
bash = bash[bash.user_id != 355]
bash = bash[bash.user_id != 356] 
bash = bash[bash.user_id != 357] 
bash = bash[bash.user_id != 358] 
bash = bash[bash.user_id != 359] 
bash = bash[bash.user_id != 360]           
bash = bash[bash.user_id != 361]
bash = bash[bash.user_id != 362]
bash = bash[bash.user_id != 363] 
bash = bash[bash.user_id != 364]
bash = bash[bash.user_id != 365]
bash = bash[bash.user_id != 366]
bash = bash[bash.user_id != 367]
bash = bash[bash.user_id != 368]
bash = bash[bash.user_id != 369]
bash = bash[bash.user_id != 372]
bash = bash[bash.user_id != 375]
bash = bash[bash.user_id != 346]
bash['cohort_id'] = bash['cohort_id'].replace(0,60)
bash.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-07-20 15:37:40,/,713,60.0,136.50.11.90,2020,7,20,15,Monday,2284981082
2020-07-20 15:40:38,/,713,60.0,136.50.11.90,2020,7,20,15,Monday,2284981082
2020-07-20 16:16:06,/,714,60.0,70.123.200.67,2020,7,20,16,Monday,1182517315
2020-07-20 16:16:11,/,715,60.0,104.182.13.78,2020,7,20,16,Monday,1756761422
2020-07-20 16:16:16,/,716,60.0,70.117.16.60,2020,7,20,16,Monday,1182076988


In [65]:
len(bash.user_id.unique())

21

#### Cohort Jupyter

In [66]:
# 62,Jupiter,2020-09-21,2021-03-30,2
jupyter = df[df.cohort_id == 62]
len(jupyter.user_id.unique())

29

In [67]:
jupyter.head()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-09-21 14:20:32,/,748,62.0,99.151.201.25,2020,9,21,14,Monday,1670891801
2020-09-21 14:20:40,javascript-i,748,62.0,99.151.201.25,2020,9,21,14,Monday,1670891801
2020-09-21 14:20:49,/,749,62.0,70.94.132.38,2020,9,21,14,Monday,1180599334
2020-09-21 14:20:53,/,750,62.0,45.20.118.158,2020,9,21,14,Monday,756315806
2020-09-21 14:20:58,toc,750,62.0,45.20.118.158,2020,9,21,14,Monday,756315806


In [68]:
jupyter.tail()

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-11-02 16:48:13,javascript-i/coffee-project,763,62.0,107.192.148.199,2020,11,2,16,Monday,1807783111
2020-11-02 16:48:17,javascript-i/mapbox-api,771,62.0,172.125.226.175,2020,11,2,16,Monday,2893931183
2020-11-02 16:48:18,javascript-i/coffee-project,771,62.0,172.125.226.175,2020,11,2,16,Monday,2893931183
2020-11-02 16:48:28,javascript-i/bom-and-dom/bom,771,62.0,172.125.226.175,2020,11,2,16,Monday,2893931183
2020-11-02 16:48:47,javascript-i/bom-and-dom/bom,763,62.0,107.192.148.199,2020,11,2,16,Monday,1807783111


#### Cohort Kalypso

In [69]:
# 132,Kalypso,2020-11-02,2021-05-04,2
test = explore.cohort_id(df,2020,11,2,10)
test.tail(100)

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-11-02 10:29:17,mysql/relationships/joins,733,61.0,104.219.136.34,2020,11,2,10,Monday,1759217698
2020-11-02 10:29:19,mysql/users,704,61.0,47.234.136.72,2020,11,2,10,Monday,803899464
2020-11-02 10:29:25,mysql/relationships/indexes,733,61.0,104.219.136.34,2020,11,2,10,Monday,1759217698
2020-11-02 10:29:31,mysql/relationships,733,61.0,104.219.136.34,2020,11,2,10,Monday,1759217698
2020-11-02 10:29:36,mysql/aliases,733,61.0,104.219.136.34,2020,11,2,10,Monday,1759217698
...,...,...,...,...,...,...,...,...,...,...
2020-11-02 10:52:08,anomaly-detection/AnomalyDetectionCartoon.jpeg,678,59.0,76.201.20.193,2020,11,2,10,Monday,1288246465
2020-11-02 10:56:25,mysql/database-design,705,61.0,136.50.70.48,2020,11,2,10,Monday,2284996144
2020-11-02 10:57:48,mysql/database-design,720,0.0,174.197.1.235,2020,11,2,10,Monday,2932146667


`No Evidence of Cohort in df`

****

In [70]:
df[df.user_id == 713]

,page_viewed,user_id,cohort_id,ip,year,month,day,hour,weekday,int_ip
datetime,,,,,,,,,,
2020-07-20 15:37:40,/,713,0.0,136.50.11.90,2020,7,20,15,Monday,2284981082
2020-07-20 15:40:38,/,713,0.0,136.50.11.90,2020,7,20,15,Monday,2284981082
2020-07-21 09:17:22,toc,713,0.0,136.50.11.90,2020,7,21,9,Tuesday,2284981082
2020-07-21 09:23:42,html-css,713,0.0,136.50.11.90,2020,7,21,9,Tuesday,2284981082
2020-07-21 09:33:35,html-css/introduction,713,0.0,136.50.11.90,2020,7,21,9,Tuesday,2284981082
...,...,...,...,...,...,...,...,...,...,...
2020-11-02 14:00:03,mysql/group-by,713,0.0,136.50.11.90,2020,11,2,14,Monday,2284981082
2020-11-02 14:18:50,mysql/database-design,713,0.0,136.50.11.90,2020,11,2,14,Monday,2284981082
2020-11-02 14:18:55,java-iii,713,0.0,136.50.11.90,2020,11,2,14,Monday,2284981082


In [72]:
df[df['cohort_id']== 0] = df.cohort_id.where((df.user_id == 713), 60)

ValueError: cannot reindex from a duplicate axis

In [ ]:
df

In [ ]:
df.info()